# Libraries and usefull functions

In [3]:
# Importing libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import json
import datetime

import pprint

import time 

# Test code

In [31]:
# Opening web browser
options = Options()
options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

# Getting the the page we want to scrap
driver.get('https://www.grainger.com/category/lighting/light-bulbs-lamps/light-bulb-lamp-accessories-changers?categoryIndex=9')
driver.implicitly_wait(1)

# Extracting page main title
title_page = driver.find_element(By.CSS_SELECTOR, '.Gb4id').text
# Identification of each sections of the page. Creating an empty dictionary to save the data.
sections = driver.find_elements(By.CLASS_NAME, 'MAcbb-')
section_dict = {}

# Looping throught each section to get the information we want which are the tables
for section in sections:
    # Extracting section title
    title_section = section.find_element(By.CLASS_NAME, "sC0Aof").text
    sub_section_dict = {}
    section_dict[title_section] = sub_section_dict

    # if there is sub section, we looping throught ever sub-section to extract every table
    if section.find_elements(By.CLASS_NAME, "T8G8vu"):
        sub_sections = section.find_elements(By.CLASS_NAME, "T8G8vu")
        extra_title = ''
        for sub_section in sub_sections:

            try:
                title_sub_section = extra_title + \
                    sub_section.find_element(By.CLASS_NAME, 'SQoGqa').text
                sub_section_dict[title_sub_section] = {}

                # To load the HTML, we need to 'activate' the JS by clicking on a button of the table
                sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

                # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
                features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
                feature_list = (features.text).split('\n')

                # Getting the table associated with the sub section
                tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
                feature_dict = {}

                # Adding the date of scrapping
                current_date = datetime.datetime.now().strftime("%Y-%m-%d")

                # Going through every row of every column and creating the list of values associated with each feature
                for row in tables.find_elements(By.TAG_NAME, 'tr'):
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    for i, cell in enumerate(cells):
                        # Extract text from each cell and add to feature_dict
                        value = cell.text
                        if feature_list[i] not in feature_dict:
                            feature_dict[feature_list[i]] = []
                        feature_dict[feature_list[i]].append(value)
                        # Add date
                        if 'Date' not in feature_dict:
                            feature_dict['Date'] = []
                        feature_dict['Date'].append(current_date)

                sub_section_dict[title_sub_section] = feature_dict

            except Exception as e:
                extra_title = sub_section.find_element(
                    By.CLASS_NAME, 'SQoGqa').text + ' '
                del sub_section_dict[title_sub_section]

    else:

        section.find_element(By.CLASS_NAME, 'JxT10f').click()

        # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
        features = section.find_element(By.CLASS_NAME, 'JxT10f')
        feature_list = (features.text).split('\n')

        # Getting the table associated with the sub section
        tables = section.find_element(By.CLASS_NAME, 'cjpIYY')
        feature_dict = {}

        # Adding the date of scrapping
        current_date = datetime.datetime.now().strftime("%Y-%m-%d")

        import time  # Import the time module for adding delays

        # Looping through every row of every column and creating the list of values associated with each feature
        for row in tables.find_elements(By.TAG_NAME, 'tr'):
            cells = row.find_elements(By.TAG_NAME, 'td')
            for i, cell in enumerate(cells):
                retry_count = 0  # Initialize retry count for this cell
                MAX_RETRIES = 3  # Maximum number of retries
                while retry_count < MAX_RETRIES:
                    try:
                        value = cell.text  # Attempt to retrieve the text of the cell
                        if feature_list[i] not in feature_dict:
                            feature_dict[feature_list[i]] = []
                        feature_dict[feature_list[i]].append(value)
                        # Add date
                        if 'Date' not in feature_dict:
                            feature_dict['Date'] = []
                        feature_dict['Date'].append(current_date)
                        break  # Break out of the retry loop if successful
                    except StaleElementReferenceException:
                        # Increment retry count and add a small delay before retrying
                        retry_count += 1
                        time.sleep(1)
                else:
                    # Handle the case when max retries are reached
                    print("Max retries reached. Unable to retrieve cell text.")
        
        sub_section_dict[title_sub_section] = feature_dict


page_dict = {}
page_dict[title_page] = section_dict

pprint.pprint(page_dict)

driver.quit()

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=124.0.6367.210); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00FBC113+48259]
	(No symbol) [0x00F4CA41]
	(No symbol) [0x00E40A17]
	(No symbol) [0x00E45B58]
	(No symbol) [0x00E47652]
	(No symbol) [0x00E476D0]
	(No symbol) [0x00E80A0B]
	(No symbol) [0x00E80C9B]
	(No symbol) [0x00E77211]
	(No symbol) [0x00EA0DE4]
	(No symbol) [0x00E76E25]
	(No symbol) [0x00EA1034]
	(No symbol) [0x00EB9B9C]
	(No symbol) [0x00EA0B36]
	(No symbol) [0x00E7570D]
	(No symbol) [0x00E762CD]
	GetHandleVerifier [0x012765A3+2908435]
	GetHandleVerifier [0x012B3BBB+3159851]
	GetHandleVerifier [0x010550CB+674875]
	GetHandleVerifier [0x0105B28C+699900]
	(No symbol) [0x00F56244]
	(No symbol) [0x00F52298]
	(No symbol) [0x00F5242C]
	(No symbol) [0x00F

# Scraping the page

In [32]:
def url_scraper(url):

    # Opening web browser
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)

    # Getting the the page we want to scrap
    driver.get(url)
    driver.implicitly_wait(1)

    # Extracting page main title
    title_page = driver.find_element(By.CSS_SELECTOR, '.Gb4id').text
    # Identification of each sections of the page. Creating an empty dictionary to save the data.
    sections = driver.find_elements(By.CLASS_NAME, 'MAcbb-')
    section_dict = {}

    # Looping throught each section to get the information we want which are the tables
    for section in sections:
        # Extracting section title
        title_section = section.find_element(By.CLASS_NAME, "sC0Aof").text
        sub_section_dict = {}
        section_dict[title_section] = sub_section_dict

        # Looping throught ever sub-section to extract every table
        if section.find_elements(By.CLASS_NAME, "T8G8vu"):
            sub_sections = section.find_elements(By.CLASS_NAME, "T8G8vu")
            extra_title = ''
            for sub_section in sub_sections:

                try:
                    title_sub_section = extra_title + \
                        sub_section.find_element(By.CLASS_NAME, 'SQoGqa').text
                    sub_section_dict[title_sub_section] = {}

                    # To load the HTML, we need to 'activate' the JS by clicking on a button of the table
                    sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

                    # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
                    features = sub_section.find_element(
                        By.CLASS_NAME, 'JxT10f')
                    feature_list = (features.text).split('\n')

                    # Getting the table associated with the sub section
                    tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
                    feature_dict = {}

                    # Adding the date of scrapping
                    current_date = datetime.datetime.now().strftime("%Y-%m-%d")

                    # Going through every row of every column and creating the list of values associated with each feature
                    rows = tables.find_elements(By.TAG_NAME, 'tr')
                    for i in range(len(feature_list)):
                        feature_dict[feature_list[i]] = []
                        feature_dict['Date'] = []

                        for row in rows:
                            value = row.find_element(
                                By.XPATH, f'./td[{i+1}]').text
                            feature_dict[feature_list[i]].append(value)
                            feature_dict['Date'].append(current_date)

                    sub_section_dict[title_sub_section] = feature_dict

                except Exception as e:
                    extra_title = sub_section.find_element(
                        By.CLASS_NAME, 'SQoGqa').text + ' '
                    del sub_section_dict[title_sub_section]

        else:

            sub_section.find_element(By.CLASS_NAME, 'JxT10f').click()

            # Getting features name as they are different on each urls. Features comes in one long str thet needs to be split
            features = sub_section.find_element(By.CLASS_NAME, 'JxT10f')
            feature_list = (features.text).split('\n')

            # Getting the table associated with the sub section
            tables = sub_section.find_element(By.CLASS_NAME, 'cjpIYY')
            feature_dict = {}

            # Adding the date of scrapping
            current_date = datetime.datetime.now().strftime("%Y-%m-%d")

            # Going through every row of every column and creating the list of values associated with each feature
            rows = tables.find_elements(By.TAG_NAME, 'tr')
            for i in range(len(feature_list)):
                feature_dict[feature_list[i]] = []
                feature_dict['Date'] = []

                for row in rows:
                    value = row.find_element(By.XPATH, f'./td[{i+1}]').text
                    feature_dict[feature_list[i]].append(value)
                    feature_dict['Date'].append(current_date)

            sub_section_dict[title_sub_section] = feature_dict

    page_dict = {}
    page_dict[title_page] = section_dict

    driver.quit()

    return page_dict

# Scrapping follow up

In [33]:
def urls_scrapped(urls):
    data = []
    # urls_done has been created if the script come to a stop for whatever reason. We can look at the last link added here and restart from here.
    urls_done = []
    for url in urls:
        print(url)  # Help to follow the script live
        try:
            scraping = url_scraper(url)
            data.append(scraping)
            urls_done.append(url)
        except:  # If there is an issue on a url we can just skip it and move on
            pass

    return data, urls_done[-1]

## Defining urls we want to scrap

In [15]:
with open('urls_extractor/urls_extractor/spiders/urls.json', 'r') as file:
    datas = json.load(file)

urls_to_scrap = []

for data in datas:
    for key, value in data.items():
        urls_to_scrap.append(value)

u = urls_to_scrap[0:3]

In [36]:
results = urls_scrapped(urls_to_scrap)
scraping = results[0]
urls_processed = results[1]

https://www.grainger.com/category/lighting/light-bulbs-lamps/circular-light-bulbs-lamps?categoryIndex=8
https://www.grainger.com/category/lighting/light-bulbs-lamps/light-bulb-lamp-accessories-changers?categoryIndex=9
https://www.grainger.com/category/lighting/light-bulbs-lamps/u-bend-light-bulbs-lamps?categoryIndex=7
https://www.grainger.com/category/lighting/light-bulbs-lamps/linear-light-bulbs-lamps?categoryIndex=1
https://www.grainger.com/category/lighting/light-bulbs-lamps/spot-reflector-flood-light-bulbs-lamps?categoryIndex=3
https://www.grainger.com/category/lighting/light-bulbs-lamps/plug-in-cfl-led-light-bulbs-lamps?categoryIndex=6
https://www.grainger.com/category/lighting/light-bulbs-lamps/high-output-hid-led-light-bulbs-lamps?categoryIndex=4
https://www.grainger.com/category/lighting/light-bulbs-lamps/standard-decorative-light-bulbs-lamps?categoryIndex=2
https://www.grainger.com/category/lighting/light-bulbs-lamps/miniature-light-bulbs-lamps/spade-wire-terminal-base-miniatu

# Saving to json

In [19]:
def json__saving(page_dict):
    json_string = json.dumps(page_dict)
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    with open(f'scrapings/{current_date}_mon_fichier.json', 'w') as datas:
        datas.write(json_string)

In [37]:
json__saving(scraping)

# Here we should save scraping to json and save this json on the data base.